In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-02-21 22:54:21.123570: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 22:54:21.266031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740196461.318948   25049 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740196461.339482   25049 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 22:54:21.456703: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4050 Laptop GPU. Max memory: 5.646 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Phi Inference

In [3]:
pregunta = "¿Cuál es el tratamiento de primera línea para la hipertensión?"

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": pregunta},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
respuesta = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0][len(pregunta):]
respuesta

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


' El tratamiento de primera línea para la hipertensión generalmente implica cambios en el estilo de vida, como dieta, ejercicio y reducción del consumo de alcohol y tabaco.\n\nSi los cambios en el estilo de vida no son suficientes para controlar la presión arterial, se pueden recetar medicamentos antihipertensivos. Los medicamentos antihipertensivos más comúnmente utilizados incluyen:\n\n1. Inhibidores de la ECA (por ejemplo, enalapril, lisinopril)\n2. Bloqueadores de los receptores de angiotensina II (ARBs, por ejemplo, losartán, valsartán)\n3. Bloqueadores de los canales de calcio (por ejemplo, amlodipino, nifedipino)\n4. Diuréticos (por ejemplo, hidroclorotiazida, metolazona)\n\nLa elección del medicamento y la dosis se basan en la condición clínica del paciente, la tolerabilidad y la eficacia.\n\nEs importante notar'

## Testing with deep eval

In [5]:
import os
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("API key loaded successfully!")
else:
    print("API key not found. Make sure it's set in your environment.")

API key loaded successfully!


## PARTE 1: Generar un json (lista de diccionarios) de preguntas y respuestas esperadas utilizando la API de ChatGPT

**Creando la funcion**

In [6]:
def generate_qa_pairs(num_pares=5):
    prompt = (f"Genera {num_pares} pares de preguntas y respuestas sobre medicina (para probar el conocimiento medico de otro LLM), "
              "en formato JSON. La estructura debe ser un objeto con una lista de pares bajo cualquier clave, "
              "donde cada objeto tenga 'input' y 'expected_output'. Ejemplo: "
              """{"qa_pairs": [{"input": "...", "expected_output": "..."}]}""")
    
    client = openai.OpenAI()
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Eres un medico experto. Genera solo JSON válido con lista bajo cualquier clave."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=2000,
            response_format={"type": "json_object"}
        )

        contenido = response.choices[0].message.content.strip()
        contenido = contenido.replace("```json", "").replace("```", "").strip()
        
        # Parsear JSON
        parsed_data = json.loads(contenido)
        
        # Buscar la primera clave que contenga una lista de objetos
        pairs = None
        if isinstance(parsed_data, dict):
            for key, value in parsed_data.items():
                if isinstance(value, list):
                    pairs = value
                    break
        
        # Validar estructura
        if not pairs:
            raise ValueError("No se encontró lista válida en el JSON")
            
        if not isinstance(pairs, list):
            raise ValueError("El formato raíz no contiene una lista")
            
        for item in pairs:
            if not all(key in item for key in ['input', 'expected_output']):
                raise ValueError("Faltan claves requeridas en los objetos")
                
        return pairs

    except openai.APIError as e:
        print(f"🚨 Error de API: {e.status_code} - {e.message}")
        return None
    except json.JSONDecodeError as e:
        print(f"📄 Error JSON (Posición {e.pos}): {e.msg}")
        print(f"Contenido problemático:\n{contenido[:500]}...")
        return None
    except ValueError as e:
        print(f"🔍 Error de validación: {str(e)}")
        print(f"Estructura recibida:\n{json.dumps(parsed_data, indent=2, ensure_ascii=False)[:500]}...")
        return None
    except Exception as e:
        print(f"⚠️ Error inesperado: {type(e).__name__} - {str(e)}")
        return None

# Funciones para cargar dataset, agregarle rows o guardarlo


In [7]:
dataset_path = 'data/qa_pairs.json'


In [8]:
import json
import os

def cargar_pairs_del_json(archivo=dataset_path):
    """Carga el dataset existente desde un archivo JSON"""
    try:
        if os.path.exists(archivo):
            with open(archivo, 'r', encoding='utf-8') as f:
                return json.load(f)
        return []
    except Exception as e:
        print(f"Error al cargar el dataset: {e}")
        return []

def guardar_pairs_al_json(dataset, archivo = dataset_path):
    """Guarda el dataset completo en un archivo JSON"""
    try:
        with open(archivo, 'w', encoding='utf-8') as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)
        return True
    except Exception as e:
        print(f"Error al guardar el dataset: {e}")
        return False

def agregar_pairs_al_json(nuevos_ejemplos, archivo= dataset_path):
    """
    Agrega nuevos ejemplos al dataset existente y guarda el resultado en disco.
    
    Parámetros:
        nuevos_ejemplos (list): Lista de nuevos ejemplos a agregar.
        archivo (str): Ruta del archivo JSON donde se guardará el dataset actualizado.
    
    Retorna:
        bool: True si la operación fue exitosa, False en caso de error.
    """
    try:
        # 1. Validar nuevos_ejemplos
        if not nuevos_ejemplos or not isinstance(nuevos_ejemplos, list):
            raise ValueError("Los nuevos ejemplos deben ser una lista no vacía")
        
        # 2. Cargar dataset existente
        dataset_existente = cargar_dataset(archivo)
        if not isinstance(dataset_existente, list):
            raise TypeError("El dataset existente no es una lista válida")
        
        # 3. Combinar datasets
        dataset_completo = dataset_existente + nuevos_ejemplos
        
        # 4. Guardar en disco
        if guardar_dataset(dataset_completo, archivo):
            print(f"✅ Añadidos {len(nuevos_ejemplos)} registros (Total: {len(dataset_completo)})")
            return True
        
        return False
        
    except ValueError as e:
        print(f"🔴 Error de validación: {str(e)}")
        return False
    except TypeError as e:
        print(f"🔴 Error de tipo: {str(e)}")
        return False
    except Exception as e:
        print(f"🔥 Error crítico: {str(e)}")
        return False

**Generar mas datos**

In [9]:
generar_mas = False
cantidad_a_generar = 3

# 1. Generar nuevos datos
if generar_mas:
    extra_pairs = generate_qa_pairs(cantidad_a_generar)  # Tu función original}
    print(extra_pairs)


**Agregar los datos generados al archivo json local**    
Generar esta separado de agregar en caso de que los casos generados no sean satisfactorios

In [10]:
agregar_datos = False
# 2. Agregar al dataset existente
if agregar_datos:
    new_total_qa_pairs = agregar_pairs_al_json(extra_pairs)
    

Cargar el archivo json

In [12]:
# 3. Cargar dataset completo en cualquier momento
qa_pairs = cargar_pairs_del_json()

# 4. Verificar contenido
if qa_pairs:
    print(f"\Archivo JSON completo ({len(qa_pairs)} entradas)")
    print(json.dumps(qa_pairs[:2], indent=2, ensure_ascii=False))

\Archivo JSON completo (11 entradas)
[
  {
    "input": "¿Qué es el sistema endocrino y cuál es su función principal?",
    "expected_output": "El sistema endocrino es una red de glándulas y órganos que producen, almacenan y secretan hormonas. Las hormonas son sustancias químicas que llevan mensajes por todo el cuerpo, desempeñando un papel clave en la regulación de una serie de funciones corporales, como el metabolismo, el crecimiento y desarrollo, el funcionamiento del sistema inmunológico y la regulación del estado de ánimo y las emociones."
  },
  {
    "input": "¿Qué es la diabetes y cómo está relacionada con el sistema endocrino?",
    "expected_output": "La diabetes es una enfermedad crónica en la que el cuerpo no puede regular los niveles de azúcar en la sangre correctamente. Esto puede ser debido a que el páncreas no produce suficiente insulina (una hormona que ayuda a las células a absorber glucosa), como en la diabetes tipo 1, o a que el cuerpo no puede usar la insulina de m

# Ejecutar inferencia con Phi en cada entrada y guardar los resultados en un archivo local

In [15]:
path_to_unscored = "data/phi_unscored.csv"
path_to_scored = "data/phi_scored.csv"

In [19]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer


def inferir_modelo(model, tokenizer, texto_input):
    # Tokeniza el input
    inputs = tokenizer(texto_input, return_tensors="pt").to("cuda")
    # Genera la respuesta (puedes ajustar max_new_tokens y otros parámetros)
    outputs = model.generate(input_ids=inputs.input_ids, max_new_tokens=256, use_cache=True)
    # Decodifica la respuesta
    respuesta = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0][4 + len(texto_input):]
    return respuesta

# Lista para almacenar los resultados finales
resultados = []

current_number = 1
# Recorre cada caso en el dataset generado (asegúrate de que dataset_medico es una lista de diccionarios)
for caso in qa_pairs:
    print(f"Ejecutando caso: {current_number}/{len(qa_pairs)} ")
    current_number += 1
    
    pregunta = caso.get("input")
    respuesta_esperada = caso.get("expected_output")
    
    # Ejecuta inferencia con Model A
    phi_output = inferir_modelo(model, tokenizer, pregunta)
    
    # Guarda en el diccionario
    resultados.append({
        "input": pregunta,
        "expected_output": respuesta_esperada,
        "phi_output": phi_output,
        # Aquí podrías agregar también el puntaje si usas una métrica (más adelante)
    })

# Convierte la lista de resultados en un DataFrame de pandas y guarda en CSV
df = pd.DataFrame(resultados)
df.to_csv(path_to_unscored, index=False)
print("Resultados guardados en 'phi_unscored.csv'")


Ejecutando caso: 1/11 
Ejecutando caso: 2/11 
Ejecutando caso: 3/11 
Ejecutando caso: 4/11 
Ejecutando caso: 5/11 
Ejecutando caso: 6/11 
Ejecutando caso: 7/11 
Ejecutando caso: 8/11 
Ejecutando caso: 9/11 
Ejecutando caso: 10/11 
Ejecutando caso: 11/11 
Resultados guardados en 'phi_unscored.csv'


In [17]:
df
#df["phi_output"][1]

,input,expected_output,phi_output
0,¿Qué es el sistema endocrino y cuál es su func...,El sistema endocrino es una red de glándulas y...,\n\nEl sistema endocrino es una colección de g...
1,¿Qué es la diabetes y cómo está relacionada co...,La diabetes es una enfermedad crónica en la qu...,\n\nLa diabetes es una enfermedad crónica que ...
2,¿Qué hormonas produce la glándula tiroides y q...,La glándula tiroides produce principalmente do...,\n\n# Answer\nLa glándula tiroides produce pri...
3,¿Cómo se diagnostica y se trata el hipotiroidi...,El hipotiroidismo se diagnostica generalmente ...,"\n\nEl hipotiroidismo, una condición en la que..."
4,¿Qué es el cortisol y cómo está relacionado co...,El cortisol es una hormona producida por las g...,\n\nEl cortisol es una hormona producida por l...
5,¿Cuál es la función principal de los glóbulos ...,Transportar oxígeno desde los pulmones a los t...,\n\n# Answer\nLa función principal de los glób...
6,¿Qué es la hipertensión arterial?,Es una condición médica en la que la presión a...,"\n\nHipertensión arterial, comúnmente conocida..."
7,¿Cuál es el principal órgano responsable de la...,El páncreas.,\n\n# Answer\nEl principal órgano responsable ...
8,¿Cuál es la función principal de los glóbulos ...,Transportar oxígeno desde los pulmones hacia l...,\n\n# Answer\nLa función principal de los glób...
9,¿Qué es la hipertensión arterial?,Es una condición médica en la que la presión a...,"\n\nHipertensión arterial, comúnmente conocida..."


## Creating the evaluation metric (prompt)

In [18]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

medical_correctness_metric = GEval(
    name="MedicalCorrectness",
    criteria="Determinar si la respuesta médica proporcionada es correcta, clínica y segura comparándola con la respuesta esperada y estándares médicos.",
    evaluation_steps=[
        "Verificar que los hechos y datos médicos en la 'actual output' sean correctos según la 'expected output' y conocimiento médico estándar.",
        "Evaluar si se utiliza la terminología médica apropiada y se siguen guías clínicas reconocidas.",
        "Penalizar la omisión de detalles importantes que puedan afectar la seguridad o la precisión clínica.",
        "Permitir ligeras diferencias en la redacción siempre y cuando el contenido clínico sea correcto y seguro."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT, 
        LLMTestCaseParams.ACTUAL_OUTPUT, 
        LLMTestCaseParams.EXPECTED_OUTPUT
    ],
    threshold=0.7,  # Este umbral se utiliza para determinar un "pass" o "fail", pero el score en sí es un valor continuo.
    model="gpt-4o-mini"
)

In [27]:
import pandas as pd
from deepeval.test_case import LLMTestCase


# Load your dataset
df = pd.read_csv('path_to_unscored.csv')  # Replace with your actual dataset path

# Create columns to store results
df['phi_score'] = None
df['result'] = None

# Iterate through each row
for index, row in df.iterrows():
    try:
        # Create test case from current row
        test_case = LLMTestCase(
            input=row['input'],
            actual_output=row['phi_output'],
            expected_output=row['expected_output']
        )
        
        # Measure medical correctness
        medical_correctness_metric.measure(test_case)
        
        # Store results
        df.at[index, 'phi_score'] = medical_correctness_metric.score
        df.at[index, 'result'] = (
            "PASÓ" if medical_correctness_metric.score >= medical_correctness_metric.threshold 
            else "FALLÓ"
        )
        
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        df.at[index, 'result'] = "ERROR"

# Save updated dataset (optional)
df.to_csv('phi_scored.csv', index=False)

print("Evaluation completed. First 5 rows:")
print(df.head())

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Evaluation completed. First 5 rows:
                                               input  \
0  ¿Qué es el sistema endocrino y cuál es su func...   
1  ¿Qué es la diabetes y cómo está relacionada co...   
2  ¿Qué hormonas produce la glándula tiroides y q...   
3  ¿Cómo se diagnostica y se trata el hipotiroidi...   
4  ¿Qué es el cortisol y cómo está relacionado co...   

                                     expected_output  \
0  El sistema endocrino es una red de glándulas y...   
1  La diabetes es una enfermedad crónica en la qu...   
2  La glándula tiroides produce principalmente do...   
3  El hipotiroidismo se diagnostica generalmente ...   
4  El cortisol es una hormona producida por las g...   

                                          phi_output phi_score result  
0  \n\nEl sistema endocrino es una colección de g...  0.734087   PASÓ  
1  \n\nLa diabetes es una enfermedad crónica que ...  0.622945  FALLÓ  
2  \n\n# Answer\nLa glándula tiroides produce pri...  0.193507  FALLÓ  
3 

In [28]:
df

,input,expected_output,phi_output,phi_score,result
0,¿Qué es el sistema endocrino y cuál es su func...,El sistema endocrino es una red de glándulas y...,\n\nEl sistema endocrino es una colección de g...,0.734087,PASÓ
1,¿Qué es la diabetes y cómo está relacionada co...,La diabetes es una enfermedad crónica en la qu...,\n\nLa diabetes es una enfermedad crónica que ...,0.622945,FALLÓ
2,¿Qué hormonas produce la glándula tiroides y q...,La glándula tiroides produce principalmente do...,\n\n# Answer\nLa glándula tiroides produce pri...,0.193507,FALLÓ
3,¿Cómo se diagnostica y se trata el hipotiroidi...,El hipotiroidismo se diagnostica generalmente ...,"\n\nEl hipotiroidismo, una condición en la que...",0.633613,FALLÓ
4,¿Qué es el cortisol y cómo está relacionado co...,El cortisol es una hormona producida por las g...,\n\nEl cortisol es una hormona producida por l...,0.65,FALLÓ


In [39]:
pd.set_option('display.max_colwidth', 1000)
df.iloc[2]

input                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ¿Qué hormonas produce la glándula tiroides y qué funciones desempeñan?
expected_output                                                                                                                                                                                 

In [32]:
phi_average_score = df["phi_score"].mean()
phi_average_score

0.566830562630641